# 1.5 금융용어사전 크롤링

## 1.5.1 전체 페이지 수 구하기

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
url = 'https://fine.fss.or.kr/fine/fnctip/fncDicary/list.do?menuNo=900021&pageIndex=1&src=&kind=&searchCnd=1&searchStr='

response = requests.get(url)
html = response.text

soup = bs(html, 'html.parser')

In [3]:
# 페이지 수 구하기
result = soup.select_one('#content > div.count-total')
page_total = int(result.text[-8:-6])
print(page_total)

54


## 1.5.2 전체 용어 크롤링

In [4]:
# 크롤링한 데이터를 저장할 DataFrame
df = pd.DataFrame(columns=['term','content'])

In [5]:
for i in range(page_total):
    url = 'https://fine.fss.or.kr/fine/fnctip/fncDicary/list.do?menuNo=900021&pageIndex='+str(i+1)+'&src=&kind=&searchCnd=1&searchStr='

    response = requests.get(url)
    html = response.text
    
    soup = bs(html, 'html.parser')
    results = soup.select('#content > div.bd-list.result-list > dl')

    
    for result in results:
        title_split = result.find('dt').text.split('.')
        
        # index
        index = '{0:03d}'.format(int(title_split[0].replace('\r','').replace('\n','').replace('\t','')))
        
        # term
        term = title_split[1].replace('\r','').replace('\n','').replace('\t','')
                
        # content
        content = result.find('dd').text

        # DataFrame에 저장
        df.loc[index] = [term, content]

In [6]:
df

,term,content
538,휴면예금[Dormant Deposit],"은행 및 우체국의 요구불 예금, 저축성 예금 중에서 관련 법률에 의해 청구권의 소멸..."
537,휴면보험금[Dormant insurance],"보험계약이 실효되거나 만기되어 보험금이나 환급금 등이 발생하였음에도, 보험계약자가 ..."
536,회전결제(리볼빙)[Revolving System],회원이 사용한 카드 이용대금 가운데 회원이 신용카드사와 약속한 일정 비율(5～10...
535,회생&정리계획[RRP :Recovery & Resolution Plan],회생계획(Recovery Plan)은 위기시 금융회사의 자체 정상화 노력을 통해 건...
534,회사에 대한 조치[Measures Taken against Company],회사에 대한 조치란 감사보고서에 대한 감리결과 회사의 회계처리기준 등 법 위반행위가...
...,...,...
005,감리위원회[Accounting Oversight Deliberation Commit...,감리위원회는 외부감사 및 회계 등과 관련하여 「주식회사의 외부감사에 관한 법률」」및...
004,간주모집,"증권의 발행 당시에는 청약의 권유 대상자가 50인 미만으로 사모에 해당되지만, 발행..."
003,가장매매[wash sales],외관상으로는 증권 등의 매매가 이루어진 것으로 보이나 실질에 있어서는 권리이전을 목...
002,가용자본[Available Capital],경영상 직면한 손실위험을 보전할 목적으로 은행이 실질적으로 보유하고 있는 자기자본 ...


***

# 1.6 데이터 전처리

In [7]:
df_save = df.copy()

In [8]:
# 전처리 과정 중에 데이터 원본이 다시 필요한 경우 아래 코드 실행
df = df_save.copy()

## 1.6.1 새로운 칼럼 추가

#### term_en Column 생성 및 값 저장

In [9]:
df['term_en'] = ''

for index, row in df.iterrows():
    term = row.copy()['term']  # SettingWithCopyWarning 발생 방지를 위한 copy
    
    term_split = term.split('[')
    
    if len(term_split) == 1:
        df.loc[index, 'term'] = term_split[0].strip()
        df.loc[index, 'term_en'] = ''
    else: 
        df.loc[index, 'term'] = term_split[0].strip()
        df.loc[index, 'term_en'] = term_split[1][:-1].strip()

In [10]:
df

,term,content,term_en
538,휴면예금,"은행 및 우체국의 요구불 예금, 저축성 예금 중에서 관련 법률에 의해 청구권의 소멸...",Dormant Deposit
537,휴면보험금,"보험계약이 실효되거나 만기되어 보험금이나 환급금 등이 발생하였음에도, 보험계약자가 ...",Dormant insurance
536,회전결제(리볼빙),회원이 사용한 카드 이용대금 가운데 회원이 신용카드사와 약속한 일정 비율(5～10...,Revolving System
535,회생&정리계획,회생계획(Recovery Plan)은 위기시 금융회사의 자체 정상화 노력을 통해 건...,RRP :Recovery & Resolution Plan
534,회사에 대한 조치,회사에 대한 조치란 감사보고서에 대한 감리결과 회사의 회계처리기준 등 법 위반행위가...,Measures Taken against Company
...,...,...,...
005,감리위원회,감리위원회는 외부감사 및 회계 등과 관련하여 「주식회사의 외부감사에 관한 법률」」및...,Accounting Oversight Deliberation Committee
004,간주모집,"증권의 발행 당시에는 청약의 권유 대상자가 50인 미만으로 사모에 해당되지만, 발행...",
003,가장매매,외관상으로는 증권 등의 매매가 이루어진 것으로 보이나 실질에 있어서는 권리이전을 목...,wash sales
002,가용자본,경영상 직면한 손실위험을 보전할 목적으로 은행이 실질적으로 보유하고 있는 자기자본 ...,Available Capital


***

# 1.7 MySQL DB에 데이터 적재

## 1.7.1 DB 생성

In [11]:
import pymysql

# password 입력
password = '0000'

In [12]:
# MySQL Connection
conn = pymysql.connect(host = '127.0.0.1',
                       port = 3306,
                       user = 'root',
                       password = password,
                       charset='utf8')

In [13]:
# SQL문 실행 - DB 생성
with conn:
    with conn.cursor() as cur:
        cur.execute('CREATE DATABASE financial_terms')
        conn.commit()

## 1.7.2 테이블 생성

In [14]:
# MySQL DB Connection
conn = pymysql.connect(host = '127.0.0.1',
                       port = 3306,
                       user = 'root',
                       password = password,
                       charset='utf8',
                       database='financial_terms')

In [15]:
# 테이블 생성 SQL문
sql = '''create table terms (
            num varchar(5) primary key,
            term varchar(50),
            term_en varchar(100),
            content text);'''

In [16]:
# SQL문 실행 - 테이블 생성
with conn.cursor() as cur:
    cur.execute(sql)
    conn.commit()

## 1.7.3 데이터 저장

In [17]:
# SQL문 실행 - 데이터 삽입
with conn.cursor() as cursor:
    for index, row in df.iterrows():
        sql = '''
        INSERT INTO terms (num, term, term_en, content)
        VALUES (%s, %s, %s, %s)
        '''
        
        values = (
            index, row['term'], row['term_en'], row['content']
        )
        
        cursor.execute(sql, values)
        
    conn.commit()

In [18]:
# MySQL DB Connection 종료
conn.close()

***

# 1.8 DB 활용 - 필터링

In [19]:
# MySQL DB Connection
conn = pymysql.connect(host = '127.0.0.1',
                       port = 3306,
                       user = 'root',
                       password = password,
                       charset='utf8',
                       database='financial_terms')

## 1.8.1 필터링1 - 채권 관련 용어

In [20]:
# 필터링1 조건에 만족하는 데이터를 조회하는 SQL문
query = '''SELECT *
            FROM terms 
            WHERE content LIKE '%채권%'
            ORDER BY num;'''

In [21]:
# SQL문 실행 - 데이터 조회
with conn.cursor() as cursor:
    cursor.execute(query)
    result = cursor.fetchall()

In [22]:
# 결과 출력
print(f'조건에 일치하는 상품: 총 {len(result)}개')

result_df = pd.DataFrame(result, columns=['num', 'term', 'term_en', 'content'])
result_df

조건에 일치하는 상품: 총 55개


,num,term,term_en,content
0,005,감리위원회,Accounting Oversight Deliberation Committee,감리위원회는 외부감사 및 회계 등과 관련하여 「주식회사의 외부감사에 관한 법률」」및...
1,010,감사인선임위원회,Auditor Appointment Committee,외부감사인 선임에 있어 선임 승인의 주체가 되는 회사의 내부 기관이다. 회사의 감사...
2,018,개인신용조회회사,CB : Credit Bureau,"개인신용조회회사(Credit Bureau, 이하 CB)는 금융회사, 공공기관, 기업..."
3,050,공매도,Short Selling,공매도(Short Selling)란 소유하지 않았거나 차입한 증권을 매도하는 것을 ...
4,059,구속성예금,compensating balance,구속성예금이란 은행이 차주에 대한 여신과 연계하여 대출금액의 일정부분을 차주의 의사...
5,071,그림자금융,Shadow Banking,전통적인 은행부문이 아니면서 상업은행과 유사한 자금중개기능을 수행함에도 은행과 같은...
6,073,금리리스크,Interest Rate Risk,금리리스크란 금리가 금융회사의 재무상태에 불리하게 변동할 때 발생할 수 있는 손실을...
7,102,기업재무안정 경영참여형 사모집합투자기구(기업재무안정 PEF),Financial Stability PEF,"재무구조개선기업(부실징후기업, 회생신청기업, 재무구조개선약정 체결기업 등)의 경영정..."
8,104,기준가격,base price,펀드의 자산 가치를 나타내는 가격으로 펀드의 매수 및 매각(환매)시에 적용되며 공고...
9,115,단기매매차익 반환,Disgorgement of short-swing profit,단기매매차익 반환 제도는 주권상장법인의 임·직원(재무ㆍ회계ㆍ기획ㆍ연구개발ㆍ공시 등에...


In [23]:
print(result_df[result_df['num'] == '005']['content'][0])

감리위원회는 외부감사 및 회계 등과 관련하여 「주식회사의 외부감사에 관한 법률」」및 동법 시행령의 규정과 기타 다른 법령에서 정하고 있는 금융위원회ㆍ증권선물위원회의 업무를 전문적이고 효율적으로 수행하기 위해 증권선물위원회 내에 두는 전문심의기구이다. 감리위원회는 ① 증권선물위원회 상임위원, ② 금융위원회 자본시장국장 또는 금융위원회 3급이상 공무원 중에서 금융위원장이 지명하는 자 1명, ③ 금융감독원 전문심의위원, ④ 한국공인회계사회 회계감사위탁감리위원회위원장, ⑤ 기업회계 또는 회계감사에 관하여 전문지식과 실무경험이 있는 자로서 한국상장회사협의회 회장이 추천하는 자 1인, ⑥ 기업회계업무에 대한 전문지식을 가진 변호사 1인, ⑦ 채권자 또는 소비자보호단체 등 회계정보이용자 대표 1인, ⑧ 기업회계와 회계감사 또는 관련 법률 등에 관하여 학식과 경험이 풍부한 자 2인 등 총 9명의 위원으로 구성된다.



## 1.8.2 필터링2 - 이자 관련 용어

In [24]:
# 필터링2 조건에 만족하는 데이터를 조회하는 SQL문
query = '''SELECT *
            FROM terms 
            WHERE content LIKE '% 이자%'
            ORDER BY num;'''

In [25]:
# SQL문 실행 - 데이터 조회
with conn.cursor() as cursor:
    cursor.execute(query)
    result = cursor.fetchall()

In [26]:
# 결과 출력
print(f'조건에 일치하는 상품: 총 {len(result)}개')

result_df = pd.DataFrame(result, columns=['num', 'term', 'term_en', 'content'])
result_df

조건에 일치하는 상품: 총 22개


,num,term,term_en,content
0,118,대부업의 등록 및 금융이용자보호에 관한 법률,Act on registration of credit business and pro...,"대부업의 등록 및 감독에 관하여 필요한 사항을 규정하고, 대부업자와 여신금융기관의 ..."
1,119,대부업자,private money lender,"금전의 대부 또는 그 중개(어음할인, 양도담보 그 밖에 이와 유사한 방법에 의한 금..."
2,126,대출채권 연체기준,,대출채권 연체기준이란 연체를 인식하는 시점과 대상에 대한 기준이다. 2006년 12...
3,129,대환대출,,대환대출은 금융회사에서 대출을 받아 이전의 대출금이나 연체금을 상환하는 제도이다. ...
4,151,무수익여신,Non-Performing Loans,"무수익여신이란, 무수익산정대상여신(「은행업감독규정시행세칙」 별표 12 참고) 중 연..."
5,211,3이원방식,pricing of three profit factors,"예정위험률, 예정이자율 및 예정사업비율을 기초로 보험수리기법과 수지상등의 원칙에 따..."
6,233,순이자마진,NIM : Net Interest Margin,은행의 모든 금리부자산의 운용결과로 발생한 은행의 운용자금 한 단위당 이자순수익(운...
7,254,신용거래 융자,Loans on Margin Account,증권회사와 고객 사이의 사전 약정에 의하여 증권회사가 고객에게 주식매수 자금을 대여...
8,259,신용예탁금,credit deposits,신용예탁금은 신협의 자산운용을 지원하는 기능을 수행하기 위해 신협의 여유자금을 신협...
9,268,신용협동조합 예금자보호기금,Credit Union Depositors Protection Fund,신용협동조합의 조합원 등이 납입한 예탁금 및 적금 등의 환급을 보장하고 조합의 건전...


In [27]:
print(result_df[result_df['num'] == '118']['content'][0])

대부업의 등록 및 감독에 관하여 필요한 사항을 규정하고, 대부업자와 여신금융기관의 이자율 제한, 불법적 채권추심행위 등을 규제하는 법률로서 대부업의 건전한 발전을 도모하고 대부업자 및 여신금융기관의 거래상대방을 보호하는 것을 목적으로 2002년 8월26일 최초로 제정되어 시행되고 있다. 동법은 ①대부업 등의 정의에 관한 사항, ②대부업의 등록에 관한 사항, ③대부계약의 체결에 관한 사항, ④이자율의 제한에 관한 사항, ⑤대부업자의 검사에 관한 사항, ⑥행정조치에 관한 사항, ⑦벌칙 및 과태료에 관한 사항 등을 규정하고 있다. 동법 제정 이후 나타난 미비점을 보완하고 미등록 대부업체에 의한 불법적인 영업행태 및 불법적 채권추심행위의 근절 등을 위해 필요한 규제를 강화하고 대부업의 양성화를 지속적으로 추진하기 위하여 2016.7.25. 대형 대부업자를 중심으로 금융위원회에 등록하도록 등록 및 감독체계를 개편하였고, 등록요건에 재정적 건전성을 확인할 수 있는 기준을 마련하며, 대부업자 등의 내부통제를 강화하기 위하여 대부업 이용자 보호기준을 마련하도록 하였으며, 대기업 계열사의 우회지원을 방지하기 위해 신용공여한도를 설정하면서, 이를 위반시 부당이득에 대한 과징금을 부과하고, 과도한 대부채권 추심을 예방하고 대부업 이용자를 보호하기 위한 손해배상 보장 제도 등를 도입하였다.



In [28]:
# MySQL DB Connection 종료
conn.close()